# Image Processing in Python

**Part of the IAFIG-RMS *Python for Bioimage Analysis* Course.**

*Dr Chas Nelson*

2019-12-09 1300--1430

## Aim

To revise key tools of image processing and carry out these operations in Python.

## ILOs

* Appreciate the capabilities of `scikit-image` for image processing in a Python environment
* Apply known image processing techniques (e.g. smoothing) in a Python environment
* Recognise additional image processing techniques (e.g. deconvolution) that are possible in a Python environment
* Relate global grayscale thresholding and the logical array to segmentation and binary images

## Imports

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import ipywidgets as widgets
import IPython.display as ipyd
from skimage import io

## Data

The image we will use for the rest of this tutorial is from the Broad Bioimage Benchmark Collection data set BBBC0034v1 (https://data.broadinstitute.org/bbbc/; Thirstrup et al. 2018).

See https://data.broadinstitute.org/bbbc/BBBC034/ for the full description; however, the key points are:

* $1024 \times 1024 \times 52$ pixels
* $65 \times 65 \times 290$ nm/pixel
* 4 channels (each stored as separate files):
  * Cell membrane label (C=0)
  * Actin label (C=1)
  * DNA label (C=2)
  * Brightfield image (C=3)
  
The below cell can be run to create a local link to the data that we downloaded in the previous session. You only need to run this cell once and then you may comment it out

In [ ]:
import os

os.symlink('../01_images-in-python/assets/bbbc034v1','./assets/bbbc034v1')

## Contrast and Histogram Equalisation

* As previously mentioned, image data may not spread across the whole bit-depth (`dtype`) of an image (array).
* The submodule `skimage.exposure` provides a range of functions for spreading an image's intensity over the full range.
* The simplest approach to this is to rescale the intensity levels.

In [ ]:
# Read a multidimensional TIF file, in this case a single channel with multiple z-slices.
myStack = io.imread('./assets/bbbc034v1/AICS_12_134_C=1.tif')

# Metadata for future use later
x_pixel_size = 65  # nm
y_pixel_size = 65  # nm
z_pixel_size = 290  # nm

# Take single slice
mySlice = myStack[26,:,:]

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Create a new cell below and use the <a href='https://scikit-image.org/docs/stable/api/skimage.exposure.html#skimage.exposure.rescale_intensity'><code>skimage.exposure.rescale_intensity()</code></a> function to rescale `mySlice` from 16-bit (assume it uses the full range) to 8-bit values. Check that the np array dtype is correct. Plot the two images side by side and their histograms beneath.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Now create a new cell below and map the data to the full 16-bit range. Check that the np array dtype is correct. Plot the two images side by side (use a full 16-bit colour mapping) and their histograms beneath.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Now create a new cell below and, using the codes above and the following tutorial, create a figure howing the original image, constrast stretched image, histogram equalised image and adaptive histogram equalised image, all with their histograms. You can find the tutorial at: <a href="https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html#sphx-glr-auto-examples-color-exposure-plot-equalize-py">https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html#sphx-glr-auto-examples-color-exposure-plot-equalize-py</a>.</div>

## Image Filtering

* Many image processing tasks include filtering, either in the spatial or frequency domain.
* Again, `scitkit-image` has many of these filters built in to the submodule `scikit-image.filters`.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Using the <a href="https://scikit-image.org/docs/stable/api/skimage.filters.html"><code>skimage.filters</code></a> submodule create a figure with a crop of the original slice (256-by-256 pixels, centred) and the results of applying a Gaussian blue; median filter; unsharp mask; sobel edge filter; and Meijering neuriteness ridge oeprator of the cropped region.</div>

## Deconvolution

* One common operation in microscopy that takes place in the frequency domain is deconvolution.
* `scitkit-image.restoration` has a variety of denoising and deconvolution algorithms, including a Richardson-Lucy implementation.

In [ ]:
import psf
from skimage import transform
from skimage import exposure

sz = 11
args = {
    'shape': (sz, sz),  # size of calculated psf array in pixels
    'dims': (x_pixel_size/1000*sz, y_pixel_size/1000*sz),  # size of array in microns
    'em_wavelen': 520.0,  # emission wavelength in nanometers
    'num_aperture': 1.25,  # numerical aperture
    'refr_index': 1.333,  # refractive index
    'magnification': 100,  # magnification
}

gauss = psf.PSF(psf.GAUSSIAN | psf.EMISSION, **args)

psf_ideal = gauss.volume()

# # Display PSF before resizing for anisotropy
# f, axes = plt.subplots(2,2)
# (XZ, XY, null, ZY) = axes.flatten()
# f.suptitle("Gaussian PSF")

# ZY.imshow(psf_ideal[:,sz,:], cmap="gray", interpolation='none')
# ZY.grid(False)
# ZY.set_title("Central X-slice")

# XZ.imshow(psf_ideal[:,:,sz].T, cmap="gray", interpolation='none')
# XZ.grid(False)
# XZ.set_title("Central Y-slice")

# XY.imshow(psf_ideal[sz,:,:], cmap="gray", interpolation='none')
# XY.grid(False)
# XY.set_title("Central Z-slice")

# null.set_axis_off()  # clear unused subplot

# plt.tight_layout()
# plt.show()

# Resize for anisotropy of our image (this is a bit rough and can be done better - but it works for this example)
psf_rescaled = transform.resize(psf_ideal,
                                (np.ceil(psf_ideal.shape[0]*(x_pixel_size/z_pixel_size)),
                                 psf_ideal.shape[1],
                                 psf_ideal.shape[2]))
psf_rescaled = psf_rescaled/psf_rescaled.sum()

# # Display PSF after resizing for anisotropy
# f, axes = plt.subplots(2,2)
# (XZ, XY, null, ZY) = axes.flatten()
# f.suptitle("Gaussian PSF")

# ZY.imshow(psf_rescaled[:,psf_rescaled.shape[1]//2+1,:], cmap="gray", interpolation='none')
# ZY.grid(False)
# ZY.set_title("Central X-slice")

# XZ.imshow(psf_rescaled[:,:,psf_rescaled.shape[2]//2+1].T, cmap="gray", interpolation='none')
# XZ.grid(False)
# XZ.set_title("Central Y-slice")

# XY.imshow(psf_rescaled[psf_rescaled.shape[0]//2+1,:,:], cmap="gray", interpolation='none')
# XY.grid(False)
# XY.set_title("Central Z-slice")

# null.set_axis_off()  # clear unused subplot

# plt.tight_layout()
# plt.show()

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Using the <a href="https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.richardson_lucy"><code>skimage.restoration.richardson_lucy</code></a> function, and in a new cell, deconvolve our 3D image for channel 1 (GFP) with the PSF defined above. Display a region of the central slice before and after convolution.</div>

## Segmentation

* Here we must introduce the Python concept of Boolean or logical values: i.e. True and False
* True and False can be represented in arrays of `dtype` 'logical' or as arrays of 1s and 0s.
  * In both cases these are essentailly black and white images and can be displayed and processing as such
* There are two groups of thresholding algorithms available in `sciki-image`:
  1. Thresholding (found in `skimage.filters`), including Otsu and hysteresis thresholding
  2. More complex segmentation algorithms, e.g. active contours and the watershed algorithm (found in `skimage.segmentation`)

### Thresholding

* Usually we would combine thresholding with pre-processing, e.g. noise reduction or deconvolution, and post-processing, e.g. morphological operations to fill holes and smooth the resulting segmentation.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Using the very helpful <a href="https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.try_all_threshold"><code>skimage.filters.try_all_threshold</code></a> function see what a single slice of our nuclei-labelled channel looks like after different thresholding approaches.</div>

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Pick the best segmentation by thresholding from your results and apply morphological (binary) closing, using <a href="https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_closing"><code>skimage.morphology.binary_closing</code></a>, to fill the small holes for a cleaner segmentation.</div>

## Extracting Regions of Interest and Features

* Once segmented, we often want to measure a variety of features of our objects.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task: </strong>In a new cell, use <a href="https://scikit-image.org/docs/stable/api/skimage.measure.html"><code>skimage.measure</code></a> to get the centroid, major and minor axis length, orientation, perimeter and intensity range for cells segmented in the previous task. Can you be sure all the detected objects are cells? Can you easily filter your results to only include those you trust?</div>

## Summary

* Appreciate the capabilities of `scikit-image` for image processing in a Python environment
* Apply known image processing techniques (e.g. smoothing) in a Python environment
* Recognise additional image processing techniques (e.g. deconvolution) that are possible in a Python environment
* Relate global grayscale thresholding and the logical array to segmentation and binary images
* Extract features of objects from segmented images